### Imports

In [35]:
# LLM
from langchain_ollama import ChatOllama

# Document loader
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embedding
from langchain_community.embeddings.ollama import OllamaEmbeddings

# Vector store
from langchain.vectorstores import SKLearnVectorStore

# Prompt
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

DATA_PATH = '../data'

In [36]:
llm = ChatOllama(model = 'llama3.1', temperature = 0)
embedding = OllamaEmbeddings(model = 'nomic-embed-text')

### Reading and chunking the document

In [37]:
loader = PyPDFDirectoryLoader(DATA_PATH)
documents = loader.load()
        
text_spitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 250,
    chunk_overlap = 0
)

chunks = text_spitter.split_documents(documents)

### Vector Store

In [38]:
vector_store = SKLearnVectorStore.from_documents(
    documents = chunks,    
    embedding = embedding
)

### Using Llama with RAG

In [48]:
prompt_template = """
    Given the following extracted parts of a long document and a question, create a final answer with references to the file name and the relevant section where the information was found ("SOURCES").
    
    If the answer cannot be determined based on the provided context, simply state that you don't know.

    ALWAYS return a "SOURCES" part in your answer.

    Use three sentence maximum and keep the answer concise:

    Question: {question}
    ====================
    {context}
    ====================
    Answer:
"""

prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ['question', 'context']
)

rag_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_store.as_retriever(k = 4),
    chain_type_kwargs = {
        'prompt': prompt,
        'document_variable_name': 'context'
    }
)

In [49]:
QUERY = 'What is the effect of the Happily Ever After Potion?'
response = rag_chain.invoke(QUERY)

In [50]:
print(f'Question: {QUERY}\n')
print(f'Answer: {response["result"]}')

Question: What is the effect of the Happily Ever After Potion?

Answer: The Happily Ever After Potion is intended to bring happiness and beauty to both the drinker and their true love, but it has an unintended effect on Fiona. The potion makes her beautiful, but also breaks a spell that was cast on her, making her unable to live as an ogre again. This causes Fiona to realize she can only break the spell by marrying her true love.

SOURCES: "Shrek" script, page 28 and 56
